In [ ]:
import spacy
import re
import os
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import sklearn
import numpy as np
import glob
import nltk
import json
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from gensim import corpora
import gensim
from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [ ]:
os.chdir('/gdrive/My Drive/drivebuddy_text_classification')

In [ ]:
def filter_query(query):
    query = query.lower()
    query = re.sub(r'[@][^\s]+', '', query)
    query = re.sub(r'pav.{0,3}bhaji', ' pavbhaji ', query)
    query = re.sub(r'[\!-\/\:-\@]+', ' ', query)
    query = re.sub('[^A-Za-z0-9\s]+', ' ', query)
    query = re.sub(r'[\t\n\r\f ]+', ' ', re.sub(r'\.', '. ', query))
    query = ' '.join([w for w in query.split() if w not in stopwords.words('english')])
    
    # print (query)
    # doc = nlp(query)
    # if len(doc) > 6:
    #     tokens = [t.text for t in doc if t.pos_ in filter_POS]
    # else:
    #     tokens = [t.text for t in doc]
    # tokens = [lemmatizer.lemmatize(t) for t in tokens]
    # filt_q = ' '.join(tokens)
    filt_q = re.sub(r'\b(n\'t|nt)\b', 'not', query)
    filt_q = re.sub(r'\'ll\b', 'will', filt_q)
    return filt_q

In [ ]:
with open('dataset_mod/pavbhaji.json', 'r') as f:
    data = json.load(f)

In [ ]:
indexed_data = {d['display_url'].split('/')[-1]: d['edge_media_to_caption']['edges'][0]['node']['text'] for d in data if d['edge_media_to_caption']['edges']}

In [ ]:
file_names0_set = set([f.split('/')[-1] for f in glob.glob('dataset/images/0/*.jpg')])
file_names1_set = set([f.split('/')[-1] for f in glob.glob('dataset/images/1/*.jpg')])

In [ ]:
data_with_labels = pd.DataFrame([{'name': name, 'text': indexed_data[name], 'label': 1 if name in file_names1_set else 0} for name in file_names0_set | file_names1_set])

In [ ]:
processed_text = data_with_labels['text'].map(filter_query)
df = pd.DataFrame({'name':data_with_labels['name'], 'text':processed_text, 'label': data_with_labels['label']})

In [ ]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [ ]:
text_data = []
for t in df['text']:
    text_data.append(prepare_text_for_lda(t))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(text_data, df['label'], test_size=0.2)

In [ ]:
dictionary = corpora.Dictionary(X_train)
corpus = [dictionary.doc2bow(text) for text in X_train]
print (corpus)

[[(0, 1), (1, 1)], [(1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1)], [(0, 2), (1, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 2), (40, 1), (41, 4), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 2), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1)], [(1, 1), (17, 1), (25, 1), (30, 1), (79, 1), (80, 1), (81, 1)], [(1, 2), (72, 2), (82, 1), (83, 2), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1)], [(1, 2), (14, 2), (15, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 

In [ ]:
# fitting an LDA model on training data for 32 topics

NUM_TOPICS = 32
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)

In [ ]:
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(8, '0.058*"pavbhaji" + 0.032*"follow" + 0.019*"streetfood" + 0.016*"sodelhi"')
(2, '0.045*"pavbhaji" + 0.018*"foodie" + 0.014*"foodlover" + 0.013*"streetfood"')
(12, '0.024*"pavbhaji" + 0.024*"bhaji" + 0.008*"mumbai" + 0.008*"yummy"')
(18, '0.046*"pavbhaji" + 0.028*"follow" + 0.016*"feature" + 0.014*"pizza"')
(14, '0.016*"smile" + 0.016*"haircolor" + 0.016*"gulab" + 0.016*"jamun"')
(5, '0.087*"pavbhaji" + 0.020*"foodie" + 0.020*"yummy" + 0.011*"foodporn"')
(24, '0.073*"pavbhaji" + 0.011*"flatbread" + 0.008*"foodlovers" + 0.007*"lover"')
(27, '0.061*"pavbhaji" + 0.021*"mumbai" + 0.012*"follow" + 0.009*"foodgasm"')
(29, '0.064*"pavbhaji" + 0.021*"foodie" + 0.017*"follow" + 0.012*"foodporn"')
(20, '0.024*"pavbhaji" + 0.020*"masaladosa" + 0.020*"samosa" + 0.020*"idlisambhar"')
(0, '0.039*"pavbhaji" + 0.028*"delhi" + 0.018*"mumbai" + 0.016*"pyaar"')
(23, '0.053*"pavbhaji" + 0.015*"chaat" + 0.015*"farali" + 0.012*"vegetable"')
(25, '0.044*"pavbhaji" + 0.043*"delhi" + 0.027*"delhifoodie" + 0

In [ ]:
new_doc = 'This vadapav rivals pavbhaji'
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print (new_doc_bow)
print (ldamodel.get_document_topics(new_doc_bow))

[(1, 1), (226, 1)]
[(0, 0.010416667), (1, 0.010416667), (2, 0.010416667), (3, 0.010416667), (4, 0.010416667), (5, 0.010416667), (6, 0.010416667), (7, 0.010416667), (8, 0.010416667), (9, 0.010416667), (10, 0.010416667), (11, 0.010416667), (12, 0.010416667), (13, 0.010416667), (14, 0.010416667), (15, 0.010416667), (16, 0.010416667), (17, 0.6770833), (18, 0.010416667), (19, 0.010416667), (20, 0.010416667), (21, 0.010416667), (22, 0.010416667), (23, 0.010416667), (24, 0.010416667), (25, 0.010416667), (26, 0.010416667), (27, 0.010416667), (28, 0.010416667), (29, 0.010416667), (30, 0.010416667), (31, 0.010416667)]


In [ ]:
res = []
for doc_bow in corpus:
    res.append(ldamodel.get_document_topics(doc_bow))

In [ ]:
# prepare the matrix with shape (num of documents, number of topics) to store the scores of each document for each topic
X_train_scores = np.zeros((len(res), NUM_TOPICS))
for i, r in enumerate(res):
    X_train_scores[i, [e[0] for e in r]] = [e[1] for e in r]

In [ ]:
X_train_scores[1]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.63255841,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.34210375,
       0.        , 0.        ])

In [ ]:
res[1]

[(9, 0.6325584), (29, 0.34210375)]

In [ ]:
# using the topic scores as features

classifier = LogisticRegression()
classifier.fit(X_train_scores, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
test_corpus = [dictionary.doc2bow(text) for text in X_test]
res = []
for doc_bow in test_corpus:
    res.append(ldamodel.get_document_topics(doc_bow))

X_test_scores = np.zeros((len(res), NUM_TOPICS))
for i, r in enumerate(res):
    X_test_scores[i, [e[0] for e in r]] = [e[1] for e in r]

predicted = classifier.predict(X)

# Model Accuracy of 60% achieved
print("Accuracy:",metrics.accuracy_score(y_test, predicted))

Accuracy: 0.6043956043956044


In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(64, 32), max_iter=10000)
clf.fit(X_train_scores, y_train)

predicted = clf.predict(X_test_scores)

# Model Accuracy of 62.6% achieved
print("Accuracy:",metrics.accuracy_score(y_test, predicted))
# print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
# print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Accuracy: 0.6263736263736264
